# Time-Domain Simulation

Time-domain simulation (TDS) solves the differential-algebraic equations (DAEs) that govern power system dynamics. This type of analysis is essential for studying transient stability, which determines whether a power system can maintain synchronism following disturbances such as faults, line trips, or generator outages.

ANDES uses the implicit trapezoidal method for numerical integration, which provides a good balance between accuracy and computational efficiency. The method is A-stable, meaning it handles stiff systems well without requiring excessively small time steps.

This tutorial covers running time-domain simulations, adding various types of disturbances, configuring simulation parameters, accessing results, and troubleshooting common issues.

## Setup

In [ ]:
import andes

andes.config_logger(stream_level=20)

## Basic Simulation Workflow

A time-domain simulation always begins with a converged power flow solution, which provides the initial steady-state operating point. From this equilibrium, the DAE solver advances time while tracking how system variables evolve in response to any disturbances.

The basic workflow consists of three steps: load the case, run power flow, then run TDS. The simulation end time is controlled by `ss.TDS.config.tf`.

In [ ]:
ss = andes.load(andes.get_case('kundur/kundur_full.xlsx'))
ss.PFlow.run()

ss.TDS.config.tf = 10  # Simulate for 10 seconds
ss.TDS.run()

The Kundur test case includes a pre-defined line trip at t=2 seconds, which triggers electromechanical oscillations between the two areas. The progress bar shows simulation advancement, and the final message confirms successful completion.

### Command Line Alternative

Time-domain simulation can be run from the command line using the `-r tds` flag. The `--tf` option sets the simulation end time.

```bash
andes run kundur_full.xlsx -r tds --tf 20
```

## Adding Disturbances

Disturbances are essential for transient stability analysis because they perturb the system from its equilibrium and allow you to observe the dynamic response. ANDES provides three main disturbance devices:

| Device | Purpose |
|--------|---------|
| `Fault` | Three-phase-to-ground bus fault |
| `Toggle` | Connect or disconnect any device |
| `Alter` | Change parameter values during simulation |

To add disturbances, you must load the case with `setup=False`, add the disturbance devices, then call `setup()` before running. This sequence is necessary because the system structure is finalized during setup.

:::{seealso}
For details on how timed events are implemented internally using `TimerParam` callbacks, see {doc}`../modeling/concepts/system-architecture`.
:::

### Three-Phase Faults

The `Fault` model applies a three-phase-to-ground fault on a bus. This is the most severe type of fault and is commonly used for stability screening. The fault is characterized by its location (bus), application time (`tf`), and clearing time (`tc`).

In [ ]:
ss_fault = andes.load(andes.get_case('kundur/kundur_full.xlsx'), setup=False)

# Disable the existing Toggle to isolate the fault effect
ss_fault.Toggle.alter('u', 1, 0)

# Add fault on bus 5: apply at t=1.0s, clear at t=1.1s
ss_fault.add('Fault', {
    'bus': 5,
    'tf': 1.0,
    'tc': 1.1
})

ss_fault.setup()
ss_fault.PFlow.run()
ss_fault.TDS.config.tf = 5
ss_fault.TDS.run()

The fault parameters control its severity and duration:

| Parameter | Description | Default |
|-----------|-------------|--------|
| `bus` | Bus index for fault location | (required) |
| `tf` | Fault application time [s] | (required) |
| `tc` | Fault clearing time [s] | (required) |
| `rf` | Fault resistance [pu] | 0 |
| `xf` | Fault reactance [pu] | 1e-6 |

For a permanent fault that is never cleared, set `tc` to a time beyond the simulation end.

In [ ]:
ss_fault.TDS.plt.plot(ss_fault.GENROU.omega, ylabel='Generator Speed [pu]')

### Line and Device Trips (Toggle)

The `Toggle` model changes the connectivity status of any device at a specified time. This is useful for simulating line trips, generator trips, load shedding, or breaker operations. Unlike faults which modify the network admittance, toggles completely remove or reconnect devices.

The following example demonstrates a line trip followed by reclosure.

In [ ]:
ss_toggle = andes.load(andes.get_case('kundur/kundur_full.xlsx'), setup=False)

# Disable existing Toggle
ss_toggle.Toggle.alter('u', 1, 0)

# Trip line at t=1.0s, reclose at t=1.5s
ss_toggle.add('Toggle', {'model': 'Line', 'dev': 'Line_5', 't': 1.0})
ss_toggle.add('Toggle', {'model': 'Line', 'dev': 'Line_5', 't': 1.5})

ss_toggle.setup()
ss_toggle.PFlow.run()
ss_toggle.TDS.config.tf = 5
ss_toggle.TDS.run()

In [ ]:
ss_toggle.TDS.plt.plot(ss_toggle.GENROU.omega, ylabel='Generator Speed [pu]')

Toggle can operate on any model type. Common applications include:

- **Generator trip**: `{'model': 'GENROU', 'dev': 'GENROU_2', 't': 1.0}`
- **Load shedding**: `{'model': 'PQ', 'dev': 'PQ_1', 't': 2.0}`
- **Transformer trip**: `{'model': 'Line', 'dev': 'Line_10', 't': 0.5}`

### Parameter Changes (Alter)

The `Alter` model changes parameter or service values at specified times during simulation. This is useful for studying the effect of setpoint changes, such as adjusting governor references or exciter voltage setpoints. The alteration is specified by the source field (`src`), the method (`=`, `+`, `-`, `*`, `/`), and the amount to apply.

In [ ]:
ss_alter = andes.load(andes.get_case('kundur/kundur_full.xlsx'), setup=False)

# Disable existing Toggle
ss_alter.Toggle.alter('u', 1, 0)

# Step change in governor setpoint at t=1.0s
ss_alter.add('Alter', {
    'model': 'TGOV1',
    'dev': 1,
    'src': 'pref0',
    't': 1.0,
    'method': '=',
    'amount': 0.8  # New setpoint value
})

ss_alter.setup()
ss_alter.PFlow.run()
ss_alter.TDS.config.tf = 10
ss_alter.TDS.run()

In [ ]:
ss_alter.TDS.plt.plot(ss_alter.GENROU.omega, ylabel='Generator Speed [pu]')

## Configuration Options

The TDS routine provides several configuration options that control simulation behavior. These can be accessed and modified through `ss.TDS.config`.

In [ ]:
ss.TDS.config

The most commonly adjusted options are:

| Option | Default | Description |
|--------|---------|-------------|
| `tf` | 20 | Simulation end time [seconds] |
| `tstep` | 1/30 | Integration time step [seconds] |
| `max_iter` | 15 | Maximum Newton iterations per step |
| `tol` | 1e-6 | Convergence tolerance |
| `fixt` | 1 | Use fixed (1) or variable (0) step size |

For simulations with fast dynamics or discontinuities, you may need to reduce the time step to maintain accuracy and stability.

## Accessing Results

Time series data from the simulation is stored in the DAE time series object `ss.dae.ts`. This object contains arrays for time, state variables, and algebraic variables at each saved time point.

### Time Series Structure

In [ ]:
# Time vector
t = ss.dae.ts.t
print(f"Time points: {len(t)}, from {t[0]:.2f} to {t[-1]:.2f} seconds")

# State variables: shape (n_steps, n_states)
print(f"State variable array shape: {ss.dae.ts.x.shape}")

# Algebraic variables: shape (n_steps, n_algebs)
print(f"Algebraic variable array shape: {ss.dae.ts.y.shape}")

### Extracting Specific Variables

To extract a specific variable's time series, you need to know its address in the state or algebraic vector. Each variable object has an `a` attribute that returns its address(es). For state variables, use `ss.dae.ts.x`; for algebraic variables, use `ss.dae.ts.y`.

In [ ]:
# Get generator speed addresses
omega_addr = ss.GENROU.omega.a
print(f"Generator speed addresses: {omega_addr}")

# Extract time series for all generators
omega_ts = ss.dae.ts.x[:, omega_addr]
print(f"Speed time series shape: {omega_ts.shape}")

In [ ]:
# Bus voltage is an algebraic variable
v_addr = ss.Bus.v.a
voltage_ts = ss.dae.ts.y[:, v_addr]
print(f"Voltage time series shape: {voltage_ts.shape}")

## Plotting Results

ANDES provides a built-in plotter accessible through `ss.TDS.plt` that simplifies visualization of simulation results. The `plot()` method accepts variable objects and automatically handles address lookup and data extraction.

In [ ]:
# Plot all generator speeds
ss.TDS.plt.plot(ss.GENROU.omega)

In [ ]:
# Plot specific generators using address indices
ss.TDS.plt.plot(ss.GENROU.omega, a=[0, 2], ylabel='Speed [pu]')

### Exporting Results

Results can be exported to CSV format for further analysis in other tools.

In [ ]:
ss.TDS.plt.export_csv('kundur_results.csv')

## Multi-Stage Simulation

For complex scenarios, you can run simulation in multiple stages with parameter changes between stages. After completing one stage, simply update the end time and call `run()` again. ANDES will continue from where it left off, preserving all state variables.

In [ ]:
ss_multi = andes.load(andes.get_case('kundur/kundur_full.xlsx'), setup=False)
ss_multi.Toggle.alter('u', 1, 0)  # Disable existing toggle
ss_multi.setup()
ss_multi.PFlow.run()

# Stage 1: Run to t=2s
ss_multi.TDS.config.tf = 2.0
ss_multi.TDS.run()

# Apply setpoint change between stages
ss_multi.TGOV1.paux0.v[0] = 0.05

# Stage 2: Continue to t=10s
ss_multi.TDS.config.tf = 10.0
ss_multi.TDS.run()

In [ ]:
# Plot shows both stages seamlessly
ss_multi.TDS.plt.plot(ss_multi.GENROU.omega)

## Output Files

TDS automatically generates output files containing simulation results:

| File | Description |
|------|-------------|
| `*_out.lst` | Variable listing with indices and names |
| `*_out.npz` | Compressed NumPy archive with time series data |

These files can be loaded later for post-processing without re-running the simulation.

In [ ]:
import numpy as np

# Load saved results
data = np.load(ss.files.npz)
print("Available arrays:", list(data.keys()))

## Troubleshooting

### Simulation Fails to Converge

If the simulation fails during integration, the Newton iteration at a particular time step did not converge within the allowed iterations. This usually indicates:

1. **Step size too large**: Reduce `tstep` to capture faster dynamics
2. **Severe disturbance**: The system may be transiently unstable
3. **Model limitations**: Some operating points may exceed model validity ranges

In [ ]:
# Typical remedies for convergence issues
ss.TDS.config.tstep = 0.001  # Smaller step size
ss.TDS.config.max_iter = 30  # More iterations allowed

### Checking Initialization

Before running a full simulation, you can initialize TDS without running to check for any initialization errors. This helps identify problems with initial conditions before committing to a long simulation.

In [ ]:
ss_check = andes.load(andes.get_case('kundur/kundur_full.xlsx'))
ss_check.PFlow.run()
ss_check.TDS.init()  # Initialize only, don't run

### Flat Run Test

A "flat run" without any disturbances verifies that the system stays at equilibrium. If variables drift during a flat run, there may be inconsistencies in the initial conditions or model parameters.

```bash
andes run case.xlsx -r tds --flat
```

## Cleanup

In [ ]:
!andes misc -C
!rm -f kundur_results.csv

## Next Steps

- {doc}`05-data-and-formats` - Loading different file formats and modifying parameters
- {doc}`06-plotting-results` - Advanced plotting and result visualization
- {doc}`07-eigenvalue-analysis` - Small-signal stability analysis